<h2>Webscraping</h2>

In [203]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

response = requests.get('https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/champions/')
response.raise_for_status()  # Check if the request was successful
soup = BeautifulSoup(response.text, 'html.parser')
links = []
for a_tag in soup.find_all('a', href=True):
    href = a_tag['href']
    if href.endswith('.json') and not href =='-1.json' and not href == '1.json':
        links.append(f"https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/champions/{href}")

def json_extract(obj, key):
    arr = []

    def extract(obj, arr, key):
        if isinstance(obj, dict):
            for k, v in obj.items():
                if k == key:
                    arr.append(v)
                elif isinstance(v, (dict,list)):
                    extract(v, arr, key)
        elif isinstance(obj, list):
            for item in obj:
                extract(item, arr, key)

        return arr
    
    values = extract(obj, arr, key)
    return values

#all_champions_Names = []
#all_champions_Info = []

champion_names = []
champion_ids = []

for url in links:
    champion_json = requests.get(url).json()
    
    id = json_extract(champion_json, 'id')
    name = json_extract(champion_json, 'name')
    champion_names.append(name[0])
    champion_ids.append(id[0])
    #champion_info = json_extract(champion_json, 'tacticalInfo')
    #champion_playstyle = json_extract(champion_json, 'playstyleInfo')

    #champion_playstyle = champion_playstyle.pop()
    #champion_info = champion_info.pop()
    #champion_name.update(champion_info)
    #champion_name.update(champion_playstyle)

    #all_champions_Info.append(champion_name)

#all_champions_df = pd.DataFrame(all_champions_Info)
championId_dict = dict(map(lambda i, j: (int(i), j), champion_ids, champion_names))
#all_champions_df.drop(['style'], axis=1, inplace=True)
championId_dict

{10: 'Kayle',
 101: 'Xerath',
 102: 'Shyvana',
 103: 'Ahri',
 104: 'Graves',
 105: 'Fizz',
 106: 'Volibear',
 107: 'Rengar',
 11: 'Master Yi',
 110: 'Varus',
 111: 'Nautilus',
 112: 'Viktor',
 113: 'Sejuani',
 114: 'Fiora',
 115: 'Ziggs',
 117: 'Lulu',
 119: 'Draven',
 12: 'Alistar',
 120: 'Hecarim',
 121: "Kha'Zix",
 122: 'Darius',
 126: 'Jayce',
 127: 'Lissandra',
 13: 'Ryze',
 131: 'Diana',
 133: 'Quinn',
 134: 'Syndra',
 136: 'Aurelion Sol',
 14: 'Sion',
 141: 'Kayn',
 142: 'Zoe',
 143: 'Zyra',
 145: "Kai'Sa",
 147: 'Seraphine',
 15: 'Sivir',
 150: 'Gnar',
 154: 'Zac',
 157: 'Yasuo',
 16: 'Soraka',
 161: "Vel'Koz",
 163: 'Taliyah',
 164: 'Camille',
 166: 'Akshan',
 17: 'Teemo',
 18: 'Tristana',
 19: 'Warwick',
 2: 'Olaf',
 20: 'Nunu & Willump',
 200: "Bel'Veth",
 201: 'Braum',
 202: 'Jhin',
 203: 'Kindred',
 21: 'Miss Fortune',
 22: 'Ashe',
 221: 'Zeri',
 222: 'Jinx',
 223: 'Tahm Kench',
 23: 'Tryndamere',
 233: 'Briar',
 234: 'Viego',
 235: 'Senna',
 236: 'Lucian',
 238: 'Zed',
 2

<h4>scraping u.gg champs wr, pickrate, banrate, tier</h4>

In [167]:
def standardizeNames(name):
    patternApos = r"([^']+)'([^']+)"
    name = re.sub(patternApos, r"\1\2", name)

    patternSpace = r'\s+'
    name = re.sub(patternSpace, '', name)

    patternPeriod = r'\.(\w)'
    name = re.sub(patternPeriod, r'\1', name)

    return name

In [181]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

response = requests.get('https://u.gg/lol/champions', headers=headers)
response.raise_for_status()  # Check if the request was successful
soup = BeautifulSoup(response.text, 'html.parser')
baseURL = "https://u.gg/"

champContainer = soup.find('div', class_='champion-home-page')
champion_data = []
if champContainer:
    links = champContainer.find_all('a')

    for link in links:
        url = baseURL + link['href']
        resp = requests.get(url, headers=headers)
        if resp.status_code == 200:
            html = resp.text
            soup1 = BeautifulSoup(html, 'html.parser')
            rankingStats = soup1.find('div', class_='champion-ranking-stats-normal')
            if rankingStats:
                text = rankingStats.text.strip()
                #scrapedData.append(rankingStats.text.strip())
                tier_pattern = r"([A-S]\+?Tier)"
                redTier_pattern = r"([A-S]\+)"
                win_rate_pattern = r"(\d+(\.\d+)?%)Win Rate"
                #r"((\d+(\.\d+)?|\d)%)Win Rate"
                rank_pattern = r"(\d+ / \d+)Rank"
                pick_rate_pattern = r"(\d+\.\d+%)Pick Rate"
                ban_rate_pattern = r"(\d+\.\d+%)Ban Rate"
                matches_pattern = r"(\d+,?\d*,?\d*)Matches"
                
                # Apply patterns
                tier = re.search(tier_pattern, text).group(1)
                win_rate = re.search(win_rate_pattern, text).group(1)
                rank = re.search(rank_pattern, text).group(1)
                pick_rate = re.search(pick_rate_pattern, text).group(1)
                ban_rate = re.search(ban_rate_pattern, text).group(1)
                matches = re.search(matches_pattern, text).group(1)

                numbers = rank.split('/')

                # Strip whitespace and convert to integers
                numerator = int(numbers[0].strip())
                denominator = int(numbers[1].strip())

                # Perform division
                rank = f"{((numerator / denominator)*100):.2f}%"

                name = soup1.find('span', class_='champion-name').text.strip()
                #name = standardizeNames(name)

                champion_data.append({
                'name' : name,
                'tier': tier,
                'win_rate': win_rate,
                'rank': rank,
                'pick_rate': pick_rate,
                'ban_rate': ban_rate,
                'matches': matches
                })



<h4> Data Table of all webscraped champions name, tier, win_rate, rank, pick_rate, ban_rate, matches </h4>

In [254]:
champion_data_df = pd.DataFrame(champion_data)
champion_data_df

,name,tier,win_rate,rank,pick_rate,ban_rate,matches
0,Aatrox,ATier,49.96%,64.41%,11.4%,16.2%,"286,783"
1,Ahri,CTier,49.66%,67.24%,8.9%,10.1%,"224,349"
2,Akali,DTier,49%,82.76%,4.8%,12.0%,"119,859"
3,Akshan,S+Tier,52.48%,5.17%,2.7%,8.6%,"67,908"
4,Alistar,STier,50.75%,27.27%,6.3%,2.3%,"158,238"
...,...,...,...,...,...,...,...
162,Zeri,CTier,49.72%,59.26%,12.0%,4.6%,"300,497"
163,Ziggs,DTier,47.88%,93.10%,1.4%,0.3%,"35,191"
164,Zilean,ATier,50.25%,40.91%,2.3%,0.7%,"57,974"
165,Zoe,ATier,50.89%,32.76%,2.5%,1.2%,"61,492"


<h2>Data Base</h2>

In [9]:
from pangres import upsert
from sqlalchemy import text, create_engine

db_username = "postgres"
db_password = "123123123"
db_host = "localhost"
db_port = "5432"
db_name = "lol_analytics"

def create_db_connection_string(db_username, db_password, db_host, db_port, db_name):
        connection_url = 'postgresql+psycopg2://'+db_username+':'+db_password+'@'+db_host+':'+db_port+'/'+db_name
        return connection_url

conn_url = create_db_connection_string(db_username, db_password, db_host, db_port, db_name)

db_engine = create_engine(conn_url, pool_recycle=3600)

connection = db_engine.connect()

<h2>Data Collection</h2>

<h4> API functions </h4>

In [186]:
import requests
import pandas as pd
api_key = "RGAPI-97d68477-a8c3-4e1f-89c8-079841d173a9"

def get_puuid_fromSummonerId(summonerId):
    puuid = requests.get(f"https://na1.api.riotgames.com/lol/summoner/v4/summoners/{summonerId}?api_key={api_key}").json()['puuid']
    return puuid

def getLeaderBoard(league):
    get_top50_url = f"https://na1.api.riotgames.com/lol/league/v4/{league}leagues/by-queue/RANKED_SOLO_5x5?api_key={api_key}"
    return requests.get(get_top50_url).json()

def get_matches(puuid):
    url = f"https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?queue=420&start=0&count=20&api_key={api_key}"
    return requests.get(url).json()

def get_matchData(matchId):
    url = f"https://americas.api.riotgames.com/lol/match/v5/matches/{matchId}?api_key={api_key}"
    return requests.get(url).json()

<h4> Gets Top Players of LOL for match retrieving </h4>

In [11]:
chal = getLeaderBoard('challenger')
gm = getLeaderBoard('grandmaster')
master = getLeaderBoard('master')

chal_df = pd.DataFrame(chal['entries'])
gm_df = pd.DataFrame(gm['entries'])
master_df = pd.DataFrame(master['entries'])

lb_df = pd.concat([chal_df, gm_df, master_df], ignore_index=True).sort_values('leaguePoints', ascending=False).reset_index(drop=True)
lb_df = lb_df.set_index('summonerId')

lb_df.to_sql('leaderboard', con=connection, schema='Top50SoloQ', if_exists='replace')


307

In [12]:
with db_engine.connect() as connection:
    df = pd.read_sql(text('SELECT * FROM "Top50SoloQ".leaderboard;'), connection)

df.head(300)

,summonerId,leaguePoints,rank,wins,losses,veteran,inactive,freshBlood,hotStreak
0,I_Ly9cVMnbqP3Lk_fuZbij0g57jAl1vM223hn0BY-Thlu14,2154,I,173,100,True,False,False,True
1,f-f6cnj0xVapOm-KtoQZ14gpHKZBVBu5TnajCeSPNcNna2M8,1661,I,199,137,True,False,False,False
2,ywLe0SCqv9aq3iPO_EWY9l_lnNa4_QtyMW0_w1csrlv5RhA,1551,I,188,136,True,False,False,True
3,xH7x54xRGwAXcxcwEa6mkANUFb2nYQuQcAEZuB7jOtadV1U,1481,I,140,86,True,False,False,False
4,cXAxF2EIp-hCfhvHDpQ6MFAb5TFHEE7aS5eFV8ZMNC_Spe...,1475,I,235,189,True,False,False,False
...,...,...,...,...,...,...,...,...,...
295,A9hwwGEslZDBJUbQ6DxtRuARy4agJvbhcamjSItlKKejMhk,723,I,99,49,False,False,True,False
296,ROXmKIkCunhuLDOemffR3hO-EPb5CKFK3jrUdRDr8BMHw5...,723,I,87,55,False,False,True,True
297,t4ho9gJ9qG9BUqKu7NpeCUCil4MLkkDDgzyb1VSqHonzDKM,723,I,143,120,True,False,False,False
298,UtrSNIsxG0DCLh3xJxPC8FARhI1t1JUPegs0G85I9jKeB1Y,721,I,119,74,False,False,True,True


<h4> Saves the Top Players PUUID and SummonerId for easy retrieval later </h4>

In [13]:
import time
puuid_list = []
count = 0
for player in df.head(300)['summonerId']:
    if count == 99:
        count = 0
        time.sleep(90) #wait for ratelimit on api
    
    puuid_list.append({'puuid' : get_puuid_fromSummonerId(player),
                       'summonerId': player})
    count += 1

puuid_df = pd.DataFrame(puuid_list)

KeyError: 'puuid'

In [14]:
puuid_df.to_sql(name='summonerdata', con=connection, schema='playerId', if_exists='replace', index=True)

NameError: name 'puuid_df' is not defined

In [16]:
with db_engine.connect() as connection:
    df1 = pd.read_sql(text('SELECT * FROM "playerId".summonerdata;'), connection)
df1

,puuid,summonerId
0,Zl5uf2ERyvcZJD_4ELxzgc9SCaltAuDOjAIiuj2sdknOA1...,I_Ly9cVMnbqP3Lk_fuZbij0g57jAl1vM223hn0BY-Thlu14
1,lWMqB3Y5AuREUMTIq2uNUzYosZFcrsGm3UwKjuW8ZdQxQS...,f-f6cnj0xVapOm-KtoQZ14gpHKZBVBu5TnajCeSPNcNna2M8
2,iOI9wt8myzTiOUZpdqYumAQo1490Axk4sFvKp1ZTtz0fui...,xH7x54xRGwAXcxcwEa6mkANUFb2nYQuQcAEZuB7jOtadV1U
3,5XqBXMuSPFUrpk807hIZ5jJ-IGj2uyOgySWq6-GrXlDcmM...,ywLe0SCqv9aq3iPO_EWY9l_lnNa4_QtyMW0_w1csrlv5RhA
4,_7r_DQhl9ENq3mc9XYlpuESPeRyv3CS2lLLHrSN-WkrDvd...,cXAxF2EIp-hCfhvHDpQ6MFAb5TFHEE7aS5eFV8ZMNC_Spe...
...,...,...
85,gIkRkkSGeoDS4LK5eluixNDmvckztMfCm9AN17_IgCWGH0...,gglipYeO0beyFFmuXxDDBH5KHZE7jgSeLEnI5q4r5NqPrj...
86,IFEwVEBnESM6acu1JrkVgalzl2Bvfw-giEkbXmvwJ1IDJy...,xJEIxbDTUNxeZuj9ykXpCSAFoQcnJmwZoen5C_DjQYIvyA...
87,T10hxgwrcpB7dg9NiK0_UiNBSGa2YP9s8Xv1pYQW_hlgsq...,0K8ZCEQIPEKJkNLQMTQvkF8DrkXCuWmj_BeAp76aAIeBUd...
88,cLxnRRh2RfKTt3-XkIOHfl_ex8vNFI6dCbplwdIa3BjTaN...,aK3-PxDQ6nV8J3yj8OHF1e-lZJVCxhA_6xftXKxzCOT7lO...


<h4> Function retrieve specific match data needed for AI modeling per match </h4>

In [184]:
def processMatchInfo(match): #FIND THE MATCHUP WINRATE
    redTeamChamps = []
    redTeamRoles = []
    blueTeamChamps = []
    blueTeamRoles = []
    outcome = 0
    participants = match['metadata']['participants']
    players = match['info']['participants']

    if match['info']['participants'][0]['win'] == False:
        if match['info']['participants'][0]['teamId'] == 200:
            outcome = 0
        else:
            outcome = 1
    else:
        if match['info']['participants'][0]['teamId'] == 200:
            outcome = 1
        else:
            outcome = 0

    def get_index(puuid):
        index = players[participants.index(puuid)]
        return index
    
    for player in participants:
        player = get_index(player)
        if player['teamId'] == 200:
            redTeamChamps.append(player['championId'])
            redTeamRoles.append(player['teamPosition'])
        else:
            blueTeamChamps.append(player['championId'])
            blueTeamRoles.append(player['teamPosition'])
    
    match_info = pd.DataFrame([{
        'outcome' : outcome,

        #red team
        'redTeam1' : redTeamChamps[0],
        'redTeam1R' : redTeamRoles[0],

        'redTeam2' : redTeamChamps[1],
        'redTeam2R' : redTeamRoles[1],

        'redTeam3' : redTeamChamps[2],
        'redTeam3R' : redTeamRoles[2],

        'redTeam4' : redTeamChamps[3],
        'redTeam4R' : redTeamRoles[3],

        'redTeam5' : redTeamChamps[4],
        'redTeam5R' : redTeamRoles[4],

        #blue team
        'blueTeam1' : blueTeamChamps[0],
        'blueTeam1R' : blueTeamRoles[0],

        'blueTeam2' : blueTeamChamps[1],
        'blueTeam2R' : blueTeamRoles[1],

        'blueTeam3' : blueTeamChamps[2],
        'blueTeam3R' : blueTeamRoles[2],

        'blueTeam4' : blueTeamChamps[3],
        'blueTeam4R' : blueTeamRoles[3],

        'blueTeam5' : blueTeamChamps[4],
        'blueTeam5R' : blueTeamRoles[4],
    }])
    return match_info

In [535]:
uniqueMatches = []
for player in df1.head(4)['puuid']:
    for match in get_matches(player):
        if match not in uniqueMatches:
            uniqueMatches.append(match)
            
uniqueMatches

['NA1_5036518415',
 'NA1_5036511826',
 'NA1_5036496169',
 'NA1_5035936299',
 'NA1_5035925053',
 'NA1_5035906397',
 'NA1_5035470687',
 'NA1_5035428371',
 'NA1_5035402815',
 'NA1_5034824451',
 'NA1_5034811770',
 'NA1_5034780669',
 'NA1_5034768874',
 'NA1_5030494093',
 'NA1_5030442871',
 'NA1_5030412958',
 'NA1_5029179586',
 'NA1_5029163020',
 'NA1_5029143841',
 'NA1_5029123902',
 'NA1_5037934629',
 'NA1_5037307147',
 'NA1_5037284418',
 'NA1_5037217508',
 'NA1_5037203555',
 'NA1_5037155954',
 'NA1_5037145233',
 'NA1_5037132916',
 'NA1_5036652312',
 'NA1_5036042211',
 'NA1_5036019687',
 'NA1_5035968193',
 'NA1_5035164262',
 'NA1_5035124591',
 'NA1_5035098470',
 'NA1_5035071141',
 'NA1_5035035400',
 'NA1_5034998719',
 'NA1_5034957165',
 'NA1_5031000548',
 'NA1_5037430766',
 'NA1_5037393498',
 'NA1_5037361769',
 'NA1_5037332586',
 'NA1_5037266198',
 'NA1_5037188685',
 'NA1_5037027131',
 'NA1_5037015452',
 'NA1_5036985454',
 'NA1_5036966329',
 'NA1_5036938922',
 'NA1_5036901293',
 'NA1_503660

In [536]:
matchall = []
all_match_df = pd.DataFrame()
for match in uniqueMatches:
    matchall.append(processMatchInfo(get_matchData(match)))

all_match_df = pd.concat(matchall)

In [540]:
all_match_df = all_match_df.replace(championId_dict)


In [541]:
all_match_df

,outcome,redTeam1,redTeam1R,redTeam2,redTeam2R,redTeam3,redTeam3R,redTeam4,redTeam4R,redTeam5,redTeam5R,blueTeam1,blueTeam1R,blueTeam2,blueTeam2R,blueTeam3,blueTeam3R,blueTeam4,blueTeam4R,blueTeam5,blueTeam5R
0,1,Kennen,TOP,Rengar,JUNGLE,Zed,MIDDLE,Kai'Sa,BOTTOM,Rakan,UTILITY,Udyr,TOP,Nidalee,JUNGLE,Galio,MIDDLE,Ezreal,BOTTOM,Seraphine,UTILITY
1,1,Camille,TOP,Fiddlesticks,JUNGLE,Corki,MIDDLE,Kai'Sa,BOTTOM,Nautilus,UTILITY,Warwick,TOP,Karthus,JUNGLE,Garen,MIDDLE,Ezreal,BOTTOM,Janna,UTILITY
2,1,Twisted Fate,TOP,Samira,BOTTOM,Ezreal,MIDDLE,Karthus,JUNGLE,Nautilus,UTILITY,Zeri,TOP,Kha'Zix,JUNGLE,Fiddlesticks,MIDDLE,Kai'Sa,BOTTOM,Alistar,UTILITY
3,0,Rumble,TOP,Viego,JUNGLE,Hwei,MIDDLE,Samira,BOTTOM,Nautilus,UTILITY,Udyr,TOP,Brand,JUNGLE,Corki,MIDDLE,Jinx,BOTTOM,Braum,UTILITY
4,1,Camille,TOP,Nunu & Willump,JUNGLE,Twisted Fate,MIDDLE,Ezreal,BOTTOM,Zilean,UTILITY,Lucian,TOP,Ekko,JUNGLE,Yone,MIDDLE,Miss Fortune,BOTTOM,Milio,UTILITY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,0,Akshan,MIDDLE,Miss Fortune,BOTTOM,Kennen,TOP,Viego,JUNGLE,Maokai,UTILITY,Graves,TOP,Nidalee,JUNGLE,Yasuo,MIDDLE,Jinx,BOTTOM,Hwei,UTILITY
72,1,Gragas,TOP,Viego,JUNGLE,Tristana,MIDDLE,Bard,UTILITY,Seraphine,BOTTOM,Gnar,TOP,Brand,JUNGLE,Corki,MIDDLE,Kai'Sa,BOTTOM,Nautilus,UTILITY
73,0,Riven,TOP,Nidalee,JUNGLE,Zoe,MIDDLE,Ezreal,BOTTOM,Zilean,UTILITY,Udyr,TOP,Kindred,JUNGLE,Hwei,MIDDLE,Zeri,BOTTOM,Lulu,UTILITY
74,1,Darius,TOP,Shaco,JUNGLE,Lucian,MIDDLE,Ezreal,BOTTOM,Blitzcrank,UTILITY,Jax,TOP,Skarner,JUNGLE,Diana,MIDDLE,Caitlyn,BOTTOM,Hwei,UTILITY


In [542]:
champion_data_df

,name,tier,win_rate,rank,pick_rate,ban_rate,matches
0,Aatrox,ATier,49.96%,64.41%,11.4%,16.2%,"286,783"
1,Ahri,CTier,49.66%,67.24%,8.9%,10.1%,"224,349"
2,Akali,DTier,49%,82.76%,4.8%,12.0%,"119,859"
3,Akshan,S+Tier,52.48%,5.17%,2.7%,8.6%,"67,908"
4,Alistar,STier,50.75%,27.27%,6.3%,2.3%,"158,238"
...,...,...,...,...,...,...,...
162,Zeri,CTier,49.72%,59.26%,12.0%,4.6%,"300,497"
163,Ziggs,DTier,47.88%,93.10%,1.4%,0.3%,"35,191"
164,Zilean,ATier,50.25%,40.91%,2.3%,0.7%,"57,974"
165,Zoe,ATier,50.89%,32.76%,2.5%,1.2%,"61,492"


In [283]:
test_df = champion_data_df[champion_data_df['name'].str.contains("nu", case=False)]
test_df['win_rate'].str.rstrip('%').astype(float)

91    50.62
Name: win_rate, dtype: float64

In [543]:
all_match_df.reset_index(drop=True, inplace=True)
def merge_champion_stats(all_match_df, champion_data_df):
    all_matchData_df = all_match_df.copy()

    pattern = re.compile(r'^(redTeam\d+|blueTeam\d+)$')
    for index, row in all_match_df.iterrows():
        for col in all_match_df.columns:
            if pattern.match(col):
                champion_name = all_match_df.loc[index, col]
                stats = champion_data_df[champion_data_df['name'] == champion_name]
                if not stats.empty:
                    for stat_col in champion_data_df.columns:
                        if stat_col != 'name' and stat_col != 'matches' :
                            all_matchData_df.loc[index, f"{col}_{stat_col}"] = stats[stat_col].values[0]

    for count in range(1, 6):
        all_matchData_df[f'redTeam{count}_win_rate'] = all_matchData_df[f'redTeam{count}_win_rate'].str.rstrip('%').astype(float)
        all_matchData_df[f'redTeam{count}_pick_rate'] = all_matchData_df[f'redTeam{count}_pick_rate'].str.rstrip('%').astype(float)
        all_matchData_df[f'redTeam{count}_ban_rate'] = all_matchData_df[f'redTeam{count}_ban_rate'].str.rstrip('%').astype(float)
        all_matchData_df[f'redTeam{count}_rank'] = all_matchData_df[f'redTeam{count}_rank'].str.rstrip('%').astype(float)

        all_matchData_df[f'blueTeam{count}_win_rate'] = all_matchData_df[f'blueTeam{count}_win_rate'].str.rstrip('%').astype(float)
        all_matchData_df[f'blueTeam{count}_pick_rate'] = all_matchData_df[f'blueTeam{count}_pick_rate'].str.rstrip('%').astype(float)
        all_matchData_df[f'blueTeam{count}_ban_rate'] = all_matchData_df[f'blueTeam{count}_ban_rate'].str.rstrip('%').astype(float)
        all_matchData_df[f'blueTeam{count}_rank'] = all_matchData_df[f'blueTeam{count}_rank'].str.rstrip('%').astype(float)

    return all_matchData_df

pd.options.display.max_columns = 100
matchup_df = merge_champion_stats(all_match_df, champion_data_df)
#all_match_df.reset_index(drop=True, inplace=True)
#all_match_df.loc[0,'redTeam1']

In [544]:
matchup_df

,outcome,redTeam1,redTeam1R,redTeam2,redTeam2R,redTeam3,redTeam3R,redTeam4,redTeam4R,redTeam5,redTeam5R,blueTeam1,blueTeam1R,blueTeam2,blueTeam2R,blueTeam3,blueTeam3R,blueTeam4,blueTeam4R,blueTeam5,blueTeam5R,redTeam1_tier,redTeam1_win_rate,redTeam1_rank,redTeam1_pick_rate,redTeam1_ban_rate,redTeam2_tier,redTeam2_win_rate,redTeam2_rank,redTeam2_pick_rate,redTeam2_ban_rate,redTeam3_tier,redTeam3_win_rate,redTeam3_rank,redTeam3_pick_rate,redTeam3_ban_rate,redTeam4_tier,redTeam4_win_rate,redTeam4_rank,redTeam4_pick_rate,redTeam4_ban_rate,redTeam5_tier,redTeam5_win_rate,redTeam5_rank,redTeam5_pick_rate,redTeam5_ban_rate,blueTeam1_tier,blueTeam1_win_rate,blueTeam1_rank,blueTeam1_pick_rate,blueTeam1_ban_rate,blueTeam2_tier,blueTeam2_win_rate,blueTeam2_rank,blueTeam2_pick_rate,blueTeam2_ban_rate,blueTeam3_tier,blueTeam3_win_rate,blueTeam3_rank,blueTeam3_pick_rate,blueTeam3_ban_rate,blueTeam4_tier,blueTeam4_win_rate,blueTeam4_rank,blueTeam4_pick_rate,blueTeam4_ban_rate,blueTeam5_tier,blueTeam5_win_rate,blueTeam5_rank,blueTeam5_pick_rate,blueTeam5_ban_rate
0,1,Kennen,TOP,Rengar,JUNGLE,Zed,MIDDLE,Kai'Sa,BOTTOM,Rakan,UTILITY,Udyr,TOP,Nidalee,JUNGLE,Galio,MIDDLE,Ezreal,BOTTOM,Seraphine,UTILITY,ATier,50.69,38.98,2.4,1.6,DTier,48.61,88.24,2.7,5.4,DTier,48.80,87.93,7.0,18.7,S+Tier,50.80,29.63,33.7,12.4,ATier,50.11,45.45,5.0,0.8,STier,51.88,5.88,3.4,2.3,S+Tier,50.17,50.98,9.2,12.7,S+Tier,52.42,8.62,3.8,1.5,DTier,49.56,74.07,25.2,15.1,ATier,50.63,34.09,4.7,1.1
1,1,Camille,TOP,Fiddlesticks,JUNGLE,Corki,MIDDLE,Kai'Sa,BOTTOM,Nautilus,UTILITY,Warwick,TOP,Karthus,JUNGLE,Garen,MIDDLE,Ezreal,BOTTOM,Janna,UTILITY,S+Tier,51.58,8.47,6.3,7.2,STier,52.00,1.96,3.2,2.7,DTier,47.91,91.38,5.9,4.2,S+Tier,50.80,29.63,33.7,12.4,DTier,49.37,72.73,13.1,20.7,ATier,51.24,13.73,2.7,1.2,DTier,49.06,76.47,3.4,13.1,STier,50.45,45.76,6.8,4.7,DTier,49.56,74.07,25.2,15.1,S+Tier,51.70,9.09,5.3,3.1
2,1,Twisted Fate,TOP,Samira,BOTTOM,Ezreal,MIDDLE,Karthus,JUNGLE,Nautilus,UTILITY,Zeri,TOP,Kha'Zix,JUNGLE,Fiddlesticks,MIDDLE,Kai'Sa,BOTTOM,Alistar,UTILITY,BTier,49.93,58.62,3.4,3.7,DTier,48.69,77.78,4.9,10.7,DTier,49.56,74.07,25.2,15.1,DTier,49.06,76.47,3.4,13.1,DTier,49.37,72.73,13.1,20.7,CTier,49.72,59.26,12.0,4.6,S+Tier,50.38,47.06,10.1,9.9,STier,52.00,1.96,3.2,2.7,S+Tier,50.80,29.63,33.7,12.4,STier,50.75,27.27,6.3,2.3
3,0,Rumble,TOP,Viego,JUNGLE,Hwei,MIDDLE,Samira,BOTTOM,Nautilus,UTILITY,Udyr,TOP,Brand,JUNGLE,Corki,MIDDLE,Jinx,BOTTOM,Braum,UTILITY,BTier,49.64,72.88,3.7,7.2,BTier,49.82,66.67,11.5,6.7,DTier,48.81,86.21,7.7,17.1,DTier,48.69,77.78,4.9,10.7,DTier,49.37,72.73,13.1,20.7,STier,51.88,5.88,3.4,2.3,DTier,47.99,88.64,2.7,15.2,DTier,47.91,91.38,5.9,4.2,S+Tier,51.36,22.22,14.9,5.9,S+Tier,51.47,13.64,6.5,3.8
4,1,Camille,TOP,Nunu & Willump,JUNGLE,Twisted Fate,MIDDLE,Ezreal,BOTTOM,Zilean,UTILITY,Lucian,TOP,Ekko,JUNGLE,Yone,MIDDLE,Miss Fortune,BOTTOM,Milio,UTILITY,S+Tier,51.58,8.47,6.3,7.2,ATier,50.62,37.25,2.4,0.4,BTier,49.93,58.62,3.4,3.7,DTier,49.56,74.07,25.2,15.1,ATier,50.25,40.91,2.3,0.7,CTier,49.56,70.37,9.8,3.2,ATier,50.48,43.14,4.1,1.4,DTier,49.42,74.14,6.4,6.7,STier,50.50,33.33,8.9,2.6,STier,50.03,50.00,7.6,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,0,Akshan,MIDDLE,Miss Fortune,BOTTOM,Kennen,TOP,Viego,JUNGLE,Maokai,UTILITY,Graves,TOP,Nidalee,JUNGLE,Yasuo,MIDDLE,Jinx,BOTTOM,Hwei,UTILITY,S+Tier,52.48,5.17,2.7,8.6,STier,50.50,33.33,8.9,2.6,ATier,50.69,38.98,2.4,1.6,BTier,49.82,66.67,11.5,6.7,ATier,50.67,29.55,2.0,0.4,DTier,48.63,86.27,7.6,4.0,S+Tier,50.17,50.98,9.2,12.7,BTier,49.68,65.52,8.4,17.8,S+Tier,51.36,22.22,14.9,5.9,DTier,48.81,86.21,7.7,17.1
72,1,Gragas,TOP,Viego,JUNGLE,Tristana,MIDDLE,Bard,UTILITY,Seraphine,BOTTOM,Gnar,TOP,Brand,JUNGLE,Corki,MIDDLE,Kai'Sa,BOTTOM,Nautilus,UTILITY,BTier,49.80,69.49,3.5,2.9,BTier,49.

<h3> Data for predicting team vs team matchup </h3>

In [332]:
def processMatchInfoGen(match):
    redTeamChamps = []
    redTeamRoles = []
    redTeamWRs = []
    redTeamPickRates = []
    redTeamBanRates = []
    redTeamTiers = []
    redTeamRanks = []

    blueTeamChamps = []
    blueTeamRoles = []
    blueTeamWRs = []
    blueTeamPickRates = []
    blueTeamBanRates = []
    blueTeamTiers = []
    blueTeamRanks = []

    outcome = 0
    participants = match['metadata']['participants']
    players = match['info']['participants']

    if match['info']['participants'][0]['win'] == False:
        if match['info']['participants'][0]['teamId'] == 200:
            outcome = 0
        else:
            outcome = 1
    else:
        if match['info']['participants'][0]['teamId'] == 200:
            outcome = 1
        else:
            outcome = 0
    #0 indicates blue side win, 1 indicates red side win

    def get_index(puuid):
        index = players[participants.index(puuid)]
        return index
    
    def get_championData(name):
        temp_df = champion_data_df[champion_data_df['name'].str.contains(name, case=False)]
        wr = temp_df['win_rate'].str.rstrip('%').astype(float).iloc[0]
        pr = temp_df['pick_rate'].str.rstrip('%').astype(float).iloc[0]
        br = temp_df['ban_rate'].str.rstrip('%').astype(float).iloc[0]
        tier = temp_df['tier'].iloc[0]
        rank = temp_df['rank'].str.rstrip('%').astype(float).iloc[0]

        return wr, pr, br, tier, rank
    
    for player in participants:
        player = get_index(player)
        if player['teamId'] == 200:
            if player['championId'] in championId_dict:
                wr, pr, br, tier, rank = get_championData(championId_dict[player['championId']])
                redTeamChamps.append(championId_dict[player['championId']])
            redTeamRoles.append(player['teamPosition'])
            redTeamWRs.append(wr)
            redTeamPickRates.append(pr)
            redTeamBanRates.append(br)
            redTeamTiers.append(tier)
            redTeamRanks.append(rank)
        else:
            if player['championId'] in championId_dict:
                wr, pr, br, tier, rank = get_championData(championId_dict[player['championId']])
                blueTeamChamps.append(championId_dict[player['championId']])
            blueTeamRoles.append(player['teamPosition'])
            blueTeamWRs.append(wr)
            blueTeamPickRates.append(pr)
            blueTeamBanRates.append(br)
            blueTeamTiers.append(tier)
            blueTeamRanks.append(rank)
    
    match_info = pd.DataFrame([{
        'outcome' : outcome,

        #red team
        'redTeam' : redTeamChamps,
        'redTeamRoles' : redTeamRoles,
        'redTeamWinRates' : redTeamWRs,
        'redTeamPickRatesm' : redTeamPickRates,
        'redTeamBanRates' : redTeamBanRates,
        'redTeamTiers' : redTeamTiers,
        'redTeamRanks' : redTeamRanks,
        #blue team
        'blueTeam' : blueTeamChamps,
        'blueTeamRoles' : blueTeamRoles,
        'blueTeamWinRates' : blueTeamWRs,
        'blueTeamPickRatesm' : blueTeamPickRates,
        'blueTeamBanRates' : blueTeamBanRates,
        'blueTeamTiers' : blueTeamTiers,
        'blueTeamRanks' : blueTeamRanks,
    }])
    return match_info

In [339]:
#idk_df = processMatchInfoGen(get_matchData('NA1_5035035400'))
#idk_df.replace(championId_dict)

matchall = []
teamvteam_df = pd.DataFrame()
for match in uniqueMatches:
    idk_df = processMatchInfoGen(get_matchData(match)).replace(championId_dict)
    matchall.append(idk_df)

teamvteam_df = pd.concat(matchall).reset_index(drop=True)

In [340]:
teamvteam_df

,outcome,redTeam,redTeamRoles,redTeamWinRates,redTeamPickRatesm,redTeamBanRates,redTeamTiers,redTeamRanks,blueTeam,blueTeamRoles,blueTeamWinRates,blueTeamPickRatesm,blueTeamBanRates,blueTeamTiers,blueTeamRanks
0,1,"[Kennen, Rengar, Zed, Kai'Sa, Rakan]","[TOP, JUNGLE, MIDDLE, BOTTOM, UTILITY]","[50.69, 48.61, 48.8, 50.8, 50.11]","[2.4, 2.7, 7.0, 33.7, 5.0]","[1.6, 5.4, 18.7, 12.4, 0.8]","[ATier, DTier, DTier, S+Tier, ATier]","[38.98, 88.24, 87.93, 29.63, 45.45]","[Udyr, Nidalee, Galio, Ezreal, Seraphine]","[TOP, JUNGLE, MIDDLE, BOTTOM, UTILITY]","[51.88, 50.17, 52.42, 49.56, 50.63]","[3.4, 9.2, 3.8, 25.2, 4.7]","[2.3, 12.7, 1.5, 15.1, 1.1]","[STier, S+Tier, S+Tier, DTier, ATier]","[5.88, 50.98, 8.62, 74.07, 34.09]"
1,1,"[Camille, Fiddlesticks, Corki, Kai'Sa, Nautilus]","[TOP, JUNGLE, MIDDLE, BOTTOM, UTILITY]","[51.58, 52.0, 47.91, 50.8, 49.37]","[6.3, 3.2, 5.9, 33.7, 13.1]","[7.2, 2.7, 4.2, 12.4, 20.7]","[S+Tier, STier, DTier, S+Tier, DTier]","[8.47, 1.96, 91.38, 29.63, 72.73]","[Warwick, Karthus, Garen, Ezreal, Janna]","[TOP, JUNGLE, MIDDLE, BOTTOM, UTILITY]","[51.24, 49.06, 50.45, 49.56, 51.7]","[2.7, 3.4, 6.8, 25.2, 5.3]","[1.2, 13.1, 4.7, 15.1, 3.1]","[ATier, DTier, STier, DTier, S+Tier]","[13.73, 76.47, 45.76, 74.07, 9.09]"
2,1,"[Twisted Fate, Samira, Ezreal, Karthus, Nautilus]","[TOP, BOTTOM, MIDDLE, JUNGLE, UTILITY]","[49.93, 48.69, 49.56, 49.06, 49.37]","[3.4, 4.9, 25.2, 3.4, 13.1]","[3.7, 10.7, 15.1, 13.1, 20.7]","[BTier, DTier, DTier, DTier, DTier]","[58.62, 77.78, 74.07, 76.47, 72.73]","[Zeri, Kha'Zix, Fiddlesticks, Kai'Sa, Alistar]","[TOP, JUNGLE, MIDDLE, BOTTOM, UTILITY]","[49.72, 50.38, 52.0, 50.8, 50.75]","[12.0, 10.1, 3.2, 33.7, 6.3]","[4.6, 9.9, 2.7, 12.4, 2.3]","[CTier, S+Tier, STier, S+Tier, STier]","[59.26, 47.06, 1.96, 29.63, 27.27]"
3,0,"[Rumble, Viego, Hwei, Samira, Nautilus]","[TOP, JUNGLE, MIDDLE, BOTTOM, UTILITY]","[49.64, 49.82, 48.81, 48.69, 49.37]","[3.7, 11.5, 7.7, 4.9, 13.1]","[7.2, 6.7, 17.1, 10.7, 20.7]","[BTier, BTier, DTier, DTier, DTier]","[72.88, 66.67, 86.21, 77.78, 72.73]","[Udyr, Brand, Corki, Jinx, Braum]","[TOP, JUNGLE, MIDDLE, BOTTOM, UTILITY]","[51.88, 47.99, 47.91, 51.36, 51.47]","[3.4, 2.7, 5.9, 14.9, 6.5]","[2.3, 15.2, 4.2, 5.9, 3.8]","[STier, DTier, DTier, S+Tier, S+Tier]","[5.88, 88.64, 91.38, 22.22, 13.64]"
4,1,"[Camille, Nunu & Willump, Twisted Fate, Ezreal...","[TOP, JUNGLE, MIDDLE, BOTTOM, UTILITY]","[51.58, 50.62, 49.93, 49.56, 50.25]","[6.3, 2.4, 3.4, 25.2, 2.3]","[7.2, 0.4, 3.7, 15.1, 0.7]","[S+Tier, ATier, BTier, DTier, ATier]","[8.47, 37.25, 58.62, 74.07, 40.91]","[Lucian, Ekko, Yone, Miss Fortune, Milio]","[TOP, JUNGLE, MIDDLE, BOTTOM, UTILITY]","[49.56, 50.48, 49.42, 50.5, 50.03]","[9.8, 4.1, 6.4, 8.9, 7.6]","[3.2, 1.4, 6.7, 2.6, 5.0]","[CTier, ATier, DTier, STier, STier]","[70.37, 43.14, 74.14, 33.33, 50.0]"
5,1,"[Camille, Ivern, Lux, Jhin, Rakan]","[TOP, JUNGLE, MIDDLE, BOTTOM, UTILITY]","[51.58, 51.91, 48.57, 50.05, 50.11]","[6.3, 1.8, 6.4, 15.2, 5.0]","[7.2, 1.0, 4.2, 3.4, 0.8]","[S+Tier, ATier, DTier, STier, ATier]","[8.47, 3.92, 81.82, 44.44, 45.45]","[Urgot, Brand, Lucian, Ezreal, Neeko]","[TOP, JUNGLE, MIDDLE, BOTTOM, UTILITY]","[51.16, 47.99, 49.56, 49.56, 49.66]","[2.3, 2.7, 9.8, 25.2, 1.6]","[0.5, 15.2, 3.2, 15.1, 0.8]","[ATier, DTier, CTier, DTier, BTier]","[13.56, 88.64, 70.37, 74.07, 65.91]"
6,1,"[Gragas, Viego, Akshan, Ezreal, Pyke]","[TOP, JUNGLE, MIDDLE, BOTTOM, UTILITY]","[49.8, 49.82, 52.48, 49.56, 50.39]","[3.5, 11.5, 2.7, 25.2, 7.1]","[2.9, 6.7, 8.6, 15.1, 20.3]","[BTier, BTier, S+Tier, DTier, S+Tier]","[69.49, 66.67, 5.17, 74.07, 38.64]","[Jax, Vi, Diana, Jhin, Bard]","[TOP, JUNGLE, MIDDLE, BOTTOM, UTILITY]","[50.1, 52.23, 49.92, 50.05, 50.2]","[6.4, 2.1, 5.3, 15.2, 3.2]","[13.1, 1.7, 2.6, 3.4, 0.4]","[S+Tier, STier, BTier, STier, ATier]","[61.02, 13.79, 58.82, 44.44, 43.18]"
7,1,"[Kennen, Lee Sin, Galio, Ezreal, Bard]","[TOP, JUNGLE, MIDDLE, BOTTOM, UTILITY]","[50.69, 49.04, 52.42, 49.56, 50.2]","[2.4, 15.9, 3.8, 25.2, 3.2]","[1.6, 13.2, 1.5, 15.

<h2> AI MODELING/TRAINING <h2>

<h4>Data preprocessing</h4>

In [396]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

tiers = ['S+Tier', 'STier', 'ATier', 'BTier', 'CTier', 'DTier']

label_encoder = LabelEncoder()

tier_encoded = label_encoder.fit_transform(tiers)
tier_mapping = dict(zip(tiers, tier_encoded))

tester_df = teamvteam_df.copy()
#tester_df.drop(columns=['redTeamRoles', 'blueTeamRoles'])

<h1> TEST </h1>

In [545]:
matchup_df

,outcome,redTeam1,redTeam1R,redTeam2,redTeam2R,redTeam3,redTeam3R,redTeam4,redTeam4R,redTeam5,redTeam5R,blueTeam1,blueTeam1R,blueTeam2,blueTeam2R,blueTeam3,blueTeam3R,blueTeam4,blueTeam4R,blueTeam5,blueTeam5R,redTeam1_tier,redTeam1_win_rate,redTeam1_rank,redTeam1_pick_rate,redTeam1_ban_rate,redTeam2_tier,redTeam2_win_rate,redTeam2_rank,redTeam2_pick_rate,redTeam2_ban_rate,redTeam3_tier,redTeam3_win_rate,redTeam3_rank,redTeam3_pick_rate,redTeam3_ban_rate,redTeam4_tier,redTeam4_win_rate,redTeam4_rank,redTeam4_pick_rate,redTeam4_ban_rate,redTeam5_tier,redTeam5_win_rate,redTeam5_rank,redTeam5_pick_rate,redTeam5_ban_rate,blueTeam1_tier,blueTeam1_win_rate,blueTeam1_rank,blueTeam1_pick_rate,blueTeam1_ban_rate,blueTeam2_tier,blueTeam2_win_rate,blueTeam2_rank,blueTeam2_pick_rate,blueTeam2_ban_rate,blueTeam3_tier,blueTeam3_win_rate,blueTeam3_rank,blueTeam3_pick_rate,blueTeam3_ban_rate,blueTeam4_tier,blueTeam4_win_rate,blueTeam4_rank,blueTeam4_pick_rate,blueTeam4_ban_rate,blueTeam5_tier,blueTeam5_win_rate,blueTeam5_rank,blueTeam5_pick_rate,blueTeam5_ban_rate
0,1,Kennen,TOP,Rengar,JUNGLE,Zed,MIDDLE,Kai'Sa,BOTTOM,Rakan,UTILITY,Udyr,TOP,Nidalee,JUNGLE,Galio,MIDDLE,Ezreal,BOTTOM,Seraphine,UTILITY,ATier,50.69,38.98,2.4,1.6,DTier,48.61,88.24,2.7,5.4,DTier,48.80,87.93,7.0,18.7,S+Tier,50.80,29.63,33.7,12.4,ATier,50.11,45.45,5.0,0.8,STier,51.88,5.88,3.4,2.3,S+Tier,50.17,50.98,9.2,12.7,S+Tier,52.42,8.62,3.8,1.5,DTier,49.56,74.07,25.2,15.1,ATier,50.63,34.09,4.7,1.1
1,1,Camille,TOP,Fiddlesticks,JUNGLE,Corki,MIDDLE,Kai'Sa,BOTTOM,Nautilus,UTILITY,Warwick,TOP,Karthus,JUNGLE,Garen,MIDDLE,Ezreal,BOTTOM,Janna,UTILITY,S+Tier,51.58,8.47,6.3,7.2,STier,52.00,1.96,3.2,2.7,DTier,47.91,91.38,5.9,4.2,S+Tier,50.80,29.63,33.7,12.4,DTier,49.37,72.73,13.1,20.7,ATier,51.24,13.73,2.7,1.2,DTier,49.06,76.47,3.4,13.1,STier,50.45,45.76,6.8,4.7,DTier,49.56,74.07,25.2,15.1,S+Tier,51.70,9.09,5.3,3.1
2,1,Twisted Fate,TOP,Samira,BOTTOM,Ezreal,MIDDLE,Karthus,JUNGLE,Nautilus,UTILITY,Zeri,TOP,Kha'Zix,JUNGLE,Fiddlesticks,MIDDLE,Kai'Sa,BOTTOM,Alistar,UTILITY,BTier,49.93,58.62,3.4,3.7,DTier,48.69,77.78,4.9,10.7,DTier,49.56,74.07,25.2,15.1,DTier,49.06,76.47,3.4,13.1,DTier,49.37,72.73,13.1,20.7,CTier,49.72,59.26,12.0,4.6,S+Tier,50.38,47.06,10.1,9.9,STier,52.00,1.96,3.2,2.7,S+Tier,50.80,29.63,33.7,12.4,STier,50.75,27.27,6.3,2.3
3,0,Rumble,TOP,Viego,JUNGLE,Hwei,MIDDLE,Samira,BOTTOM,Nautilus,UTILITY,Udyr,TOP,Brand,JUNGLE,Corki,MIDDLE,Jinx,BOTTOM,Braum,UTILITY,BTier,49.64,72.88,3.7,7.2,BTier,49.82,66.67,11.5,6.7,DTier,48.81,86.21,7.7,17.1,DTier,48.69,77.78,4.9,10.7,DTier,49.37,72.73,13.1,20.7,STier,51.88,5.88,3.4,2.3,DTier,47.99,88.64,2.7,15.2,DTier,47.91,91.38,5.9,4.2,S+Tier,51.36,22.22,14.9,5.9,S+Tier,51.47,13.64,6.5,3.8
4,1,Camille,TOP,Nunu & Willump,JUNGLE,Twisted Fate,MIDDLE,Ezreal,BOTTOM,Zilean,UTILITY,Lucian,TOP,Ekko,JUNGLE,Yone,MIDDLE,Miss Fortune,BOTTOM,Milio,UTILITY,S+Tier,51.58,8.47,6.3,7.2,ATier,50.62,37.25,2.4,0.4,BTier,49.93,58.62,3.4,3.7,DTier,49.56,74.07,25.2,15.1,ATier,50.25,40.91,2.3,0.7,CTier,49.56,70.37,9.8,3.2,ATier,50.48,43.14,4.1,1.4,DTier,49.42,74.14,6.4,6.7,STier,50.50,33.33,8.9,2.6,STier,50.03,50.00,7.6,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,0,Akshan,MIDDLE,Miss Fortune,BOTTOM,Kennen,TOP,Viego,JUNGLE,Maokai,UTILITY,Graves,TOP,Nidalee,JUNGLE,Yasuo,MIDDLE,Jinx,BOTTOM,Hwei,UTILITY,S+Tier,52.48,5.17,2.7,8.6,STier,50.50,33.33,8.9,2.6,ATier,50.69,38.98,2.4,1.6,BTier,49.82,66.67,11.5,6.7,ATier,50.67,29.55,2.0,0.4,DTier,48.63,86.27,7.6,4.0,S+Tier,50.17,50.98,9.2,12.7,BTier,49.68,65.52,8.4,17.8,S+Tier,51.36,22.22,14.9,5.9,DTier,48.81,86.21,7.7,17.1
72,1,Gragas,TOP,Viego,JUNGLE,Tristana,MIDDLE,Bard,UTILITY,Seraphine,BOTTOM,Gnar,TOP,Brand,JUNGLE,Corki,MIDDLE,Kai'Sa,BOTTOM,Nautilus,UTILITY,BTier,49.80,69.49,3.5,2.9,BTier,49.

In [546]:
idk_df = pd.DataFrame(matchup_df[['outcome', 'redTeam1', 'blueTeam1', 'redTeam1R', 
                             'redTeam1_win_rate', 'redTeam1_pick_rate', 'redTeam1_ban_rate',
                             'blueTeam1_win_rate', 'blueTeam1_pick_rate', 'blueTeam1_ban_rate']])
idk_df

,outcome,redTeam1,blueTeam1,redTeam1R,redTeam1_win_rate,redTeam1_pick_rate,redTeam1_ban_rate,blueTeam1_win_rate,blueTeam1_pick_rate,blueTeam1_ban_rate
0,1,Kennen,Udyr,TOP,50.69,2.4,1.6,51.88,3.4,2.3
1,1,Camille,Warwick,TOP,51.58,6.3,7.2,51.24,2.7,1.2
2,1,Twisted Fate,Zeri,TOP,49.93,3.4,3.7,49.72,12.0,4.6
3,0,Rumble,Udyr,TOP,49.64,3.7,7.2,51.88,3.4,2.3
4,1,Camille,Lucian,TOP,51.58,6.3,7.2,49.56,9.8,3.2
...,...,...,...,...,...,...,...,...,...,...
71,0,Akshan,Graves,MIDDLE,52.48,2.7,8.6,48.63,7.6,4.0
72,1,Gragas,Gnar,TOP,49.80,3.5,2.9,49.80,3.4,0.6
73,0,Riven,Udyr,TOP,50.81,4.2,1.5,51.88,3.4,2.3
74,1,Darius,Jax,TOP,50.55,7.4,18.3,50.10,6.4,13.1


In [547]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

all_champions = []
for champ in championId_dict.values():
    all_champions.append(champ)

label_encoder.fit(all_champions)

tiers = ['S+Tier', 'STier', 'ATier', 'BTier', 'CTier', 'DTier']

tier_encoded = label_encoder.fit_transform(tiers)
tier_mapping = dict(zip(tiers, tier_encoded))
# Fit and transform the data
idk_df['redTeam1'] = label_encoder.transform(idk_df['redTeam1'])
idk_df['blueTeam1'] = label_encoder.transform(idk_df['blueTeam1'])

x = idk_df.drop(columns=['outcome', 'redTeam1R'])
y = idk_df['outcome']


In [548]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [549]:
# Initialize the Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

In [550]:
# Train the model
rf_model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [551]:
# Make predictions
y_pred = rf_model.predict(X_test)

In [552]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.6875
